In [16]:
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [17]:
def region_selection(edges):
    '''
    定义并剪切ROI区域
    '''
    
    # 定义ROI区域
    mask = np.zeros_like(edges)
    
    # 定义ROI区域的四个顶点
    if len(edges.shape) > 2:
        channel_count = edges.shape[2]
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
    
    rows, cols = edges.shape[:2]

    #TODO: 修改调试ROI区域的四个顶点
    bottom_left = [cols * 0, rows * 0.8]
    top_left = [cols * 0.4, rows * 0.4]
    bottom_right = [cols * 0.9, rows * 0.9]
    top_right = [cols * 0.6, rows * 0.4]

    vertices = np.array([[bottom_left, top_left, top_right, bottom_right]], dtype=np.int32)

    # 填充ROI区域
    cv2.fillPoly(mask, vertices, ignore_mask_color)

    # 返回ROI区域
    masked_edges = cv2.bitwise_and(edges, mask)

    if masked_edges is None:
        print("ROI区域选择失败")
    else:
        print("ROI区域选择成功")

    return masked_edges

def hough_transform(region):
    '''
    使用霍夫变换检测直线
    '''
    
    #TODO 定义霍夫变换的参数
    rho = 1
    theta = np.pi / 180
    threshold = 50 #增加阈值以减少干扰
    min_line_length = 20
    max_line_gap = 500
    
    # 使用霍夫变换检测直线
    lines = cv2.HoughLinesP(region, rho = rho, theta = theta, threshold = threshold,
						minLineLength = min_line_length, maxLineGap = max_line_gap)
    
    # 检查返回值
    if lines is None:
        print("在Hough变换中，没有检测到任何直线")
    else:
        print(f"在Hough变换中，检测到 {len(lines)} 条直线")
    
    return lines



In [18]:
def average_slope_intercept(lines):
    '''
    计算直线的斜率和截距
    '''
    left_lines = [] #(slope, intercept)
    left_weights = [] #(length,)
    right_lines = [] #(slope, intercept)
    right_weights = [] #(length,)

    for line in lines:
        for x1, y1, x2, y2 in line:
            if x1==x2:
                continue

            # 计算直线的斜率和截距    
            slope = (y2-y1)/(x2-x1)
            intercept = y1 - slope*x1
            length = np.sqrt((y2-y1)**2+(x2-x1)**2)

            # 区分左右车道线
            # 左车道线的斜率为负
            # 右车道线的斜率为正
            # 过于接近水平的线条将被忽略
            #TODO: 修改斜率的阈值
            if abs(slope) < 0.2:
                continue

            if slope < 0:
                left_lines.append((slope, intercept))
                left_weights.append((length))
            else:
                right_lines.append((slope, intercept))
                right_weights.append((length))
            
    left_lane = np.dot(left_weights, left_lines) / np.sum(left_weights) if len(left_weights) > 0 else None
    right_lane = np.dot(right_weights, right_lines) / np.sum(right_weights) if len(right_weights) > 0 else None

    return left_lane, right_lane

def pixel_points(y1, y2, line):
    '''
    计算直线的两个端点
    '''

    if line is None:
        return None
    
    slope, intercept = line

    x1 = int((y1 - intercept) / slope)
    x2 = int((y2 - intercept) / slope)
    y1 = int(y1)
    y2 = int(y2)

    return ((x1, y1), (x2, y2))

def lane_lines(image, lines):
    '''
    从pixel_points函数获取直线的两个端点并创建车道线
    '''
    left_lane, right_lane = average_slope_intercept(lines)

    y1 = image.shape[0]
    y2 = y1 * 0.6

    left_line = pixel_points(y1, y2, left_lane)
    right_line = pixel_points(y1, y2, right_lane)

    return left_line, right_line

def draw_lane_lines(image, lines, color=[255, 0, 0], thickness=20):
    '''
    绘制车道线
    '''
    line_image = np.zeros_like(image)
    for line in lines:
        if line is not None:
            cv2.line(line_image, *line, color, thickness)
    return cv2.addWeighted(image, 1.0, line_image, 0.95, 0.0)

In [19]:
def frame_processor(image):
    '''
    将输入的图像转换为灰度图像，并进行高斯模糊处理
    '''
    # 转换为灰度图像
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    kernel_size = 5

    # 高斯模糊处理
    blur = cv2.GaussianBlur(gray, (kernel_size, kernel_size), 0)

    low_threshold = 50
    high_threshold = 150

    # 边缘检测
    edges = cv2.Canny(blur, low_threshold, high_threshold)
    #cv2.imshow('edges',edges)
    #cv2.waitKey(0)

    region = region_selection(edges)
    #print(type(region))
    cv2.imshow('region',region)
    cv2.waitKey(0)

    hough = hough_transform(region)
    #print(type(hough))
    #cv2.imshow('hough',hough)
    #cv2.waitKey(0)

    result = draw_lane_lines(image,lane_lines(image,hough))

    return result



In [20]:
image_path = './images/'

result_path = './result_images/'

# 创建一个目录用于保存结果
if not os.path.exists(result_path):
    os.mkdir(result_path)


# 获取目录下的所有文件和文件夹
image_names = os.listdir(image_path) #list

# 创建一个窗口
cv2.namedWindow('Lane Detection', cv2.WINDOW_NORMAL)

for image_name in image_names:
    # 读取图像
    image = cv2.imread(image_path + image_name)
    print(image.shape[:2])
    
    # 处理图像
    result = frame_processor(image)
    
    # 显示图像
    cv2.imshow('Lane Detection', result)

    # 保存图像
    cv2.imwrite(f'./result_images/{image_name}_result.jpg', result)
    
    # 等待按键
    cv2.waitKey(0)

cv2.destroyAllWindows()


(480, 640)
ROI区域选择成功
在Hough变换中，检测到 8 条直线
(480, 640)
ROI区域选择成功
在Hough变换中，检测到 7 条直线
(480, 640)
ROI区域选择成功
在Hough变换中，检测到 8 条直线
(480, 640)
ROI区域选择成功
在Hough变换中，检测到 8 条直线
(480, 640)
ROI区域选择成功
在Hough变换中，检测到 6 条直线
(480, 640)
ROI区域选择成功
在Hough变换中，检测到 10 条直线
(480, 640)
ROI区域选择成功
在Hough变换中，检测到 11 条直线
(480, 640)
ROI区域选择成功
在Hough变换中，检测到 6 条直线
(480, 640)
ROI区域选择成功
在Hough变换中，检测到 10 条直线
(480, 640)
ROI区域选择成功
在Hough变换中，检测到 11 条直线
(480, 640)
ROI区域选择成功
在Hough变换中，检测到 7 条直线
(480, 640)
ROI区域选择成功
在Hough变换中，检测到 10 条直线
(480, 640)
ROI区域选择成功
在Hough变换中，检测到 11 条直线
(480, 640)
ROI区域选择成功
在Hough变换中，检测到 6 条直线
(480, 640)
ROI区域选择成功
在Hough变换中，检测到 9 条直线
(480, 640)
ROI区域选择成功
在Hough变换中，检测到 10 条直线
(480, 640)
ROI区域选择成功
在Hough变换中，检测到 8 条直线
(480, 640)
ROI区域选择成功
在Hough变换中，检测到 8 条直线
(480, 640)
ROI区域选择成功
在Hough变换中，检测到 10 条直线
(480, 640)
ROI区域选择成功
在Hough变换中，检测到 7 条直线
(480, 640)
ROI区域选择成功
在Hough变换中，检测到 8 条直线
(480, 640)
ROI区域选择成功
在Hough变换中，检测到 8 条直线
(480, 640)
ROI区域选择成功
在Hough变换中，检测到 8 条直线
(480, 640)
ROI区域选择成功
在Hough变换中，检测到 18 条直线
(480, 6